In [ ]:
###Part 2-1 Twitter web scrapping Script

In [147]:
#### Import Packages ####
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import re


In [ ]:
#### Twitter Scraper ####

key_word = "Ukraine"  # Declare the key word used to search tweets
#user_name = "@mcgillu"   # Declare a user name used to search tweets
from_date = "2022-03-14" # Declare a start date
end_date = "2022-03-20"  # Declare a end date
count = 5000           # The maximum number of tweets
tweets_list_keyword = [] # A list used to store the returned results for keyword search
tweets_list_user = []    # A list used to store the retuned results for user search


In [ ]:
#### Scrape tweets from a text search query ####

command_keyword = key_word+' lang:en'+' since:'+from_date+' until:'+end_date # Define a string command for Scraper Api
print("Scraping data for keyword:",key_word)
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(command_keyword).get_items()):
    tweets_list_keyword.append([tweet.user.username, tweet.mentionedUsers]) # Append returned results to list
    if i>count:
        break;
      

In [ ]:
#### Create a dataframe from the tweets list above ####
tweets_df_keyword = pd.DataFrame(tweets_list_keyword, columns=['Source', 'Target'])
tweets_df_keyword = pd.read_csv(r"tweets_keyword_ukraine.csv")
tweets_df_keyword['Target'].fillna("nan",inplace=True)

def username_finder(x):
    username = re.findall(r'(?<=username=\')(.*?)(?=\')', x)
    return username

tweets_df_keyword['Target'] = tweets_df_keyword['Target'].apply(lambda x: username_finder(x))

tweets_df_keyword = tweets_df_keyword.explode('Target')
tweets_df_keyword = tweets_df_keyword.reset_index(drop=True)
tweets_df_keyword['Type of Content'] = np.nan

for i in range(len(tweets_df_keyword)):
    if (type(tweets_df_keyword['Target'][i]) != str ):
        tweets_df_keyword['Target'][i] = tweets_df_keyword['Source'][i]
        tweets_df_keyword['Type of Content'][i] = 'Tweet'
    else:
        tweets_df_keyword['Type of Content'][i] = 'Non-Tweet'
        
tweets_df_keyword.to_csv("tweets_keyword_ukraine_processed.csv",index=False) # Export to a csv file
print("Scraped data have been exported to the csv file")



In [ ]:
###Part 2-2 

In [98]:
data= pd.read_csv("/Users/senanagblonon/Downloads/tweets_keyword_ukraine_processed.csv") 

data

Source           Target Type of Content
0            Rneifeld          MSF_USA       Non-Tweet
1         CarolLever6      CarolLever6           Tweet
2       vondell_james        mrsorokaa       Non-Tweet
3      JuliusFlywheel  Harryslaststand       Non-Tweet
4      JuliusFlywheel  SpaceFoundation       Non-Tweet
...               ...              ...             ...
8424         1776_kat  RepTonyGonzales       Non-Tweet
8425  DerrickBerthel1  DerrickBerthel1           Tweet
8426       AlexDunnin       AlexDunnin           Tweet
8427    Sweetescapes7    Sweetescapes7           Tweet
8428    JOSEPHKAMOOTI        mrsorokaa       Non-Tweet

[8429 rows x 3 columns]

In [99]:
import networkx as nx


In [100]:
##reading our edgelist
G_graph=nx.from_pandas_edgelist(data,source='Source',target='Target')



In [101]:
###Calculating Metrics

degree_node=pd.DataFrame.from_dict(G_graph.degree())
degree_node.rename(columns={0: 'index', 1: 'Degree of a Node'})  



index  Degree of a Node
0            Rneifeld                 2
1             MSF_USA                 1
2         CarolLever6                 2
3       vondell_james                 1
4           mrsorokaa                74
...               ...               ...
7737  RepTonyGonzales                 1
7738  DerrickBerthel1                 2
7739       AlexDunnin                 2
7740    Sweetescapes7                 2
7741    JOSEPHKAMOOTI                 1

[7742 rows x 2 columns]

In [102]:
degree_centrality=pd.DataFrame.from_dict(nx.degree_centrality(G_graph), orient='index').reset_index()
degree_centrality.rename(columns={'index': 'index', 0: 'Degree Centrality Metric'})  



index  Degree Centrality Metric
0            Rneifeld                  0.000258
1             MSF_USA                  0.000129
2         CarolLever6                  0.000258
3       vondell_james                  0.000129
4           mrsorokaa                  0.009559
...               ...                       ...
7737  RepTonyGonzales                  0.000129
7738  DerrickBerthel1                  0.000258
7739       AlexDunnin                  0.000258
7740    Sweetescapes7                  0.000258
7741    JOSEPHKAMOOTI                  0.000129

[7742 rows x 2 columns]

In [105]:
betweenness=pd.DataFrame.from_dict(nx.betweenness_centrality(G_graph), orient='index').reset_index()
betweenness.rename(columns={'index': 'index', 0: 'Betweenness Metric'})  



index  Betweenness Metric
0            Rneifeld            0.000094
1             MSF_USA            0.000000
2         CarolLever6            0.000000
3       vondell_james            0.000000
4           mrsorokaa            0.014387
...               ...                 ...
7737  RepTonyGonzales            0.000000
7738  DerrickBerthel1            0.000000
7739       AlexDunnin            0.000000
7740    Sweetescapes7            0.000000
7741    JOSEPHKAMOOTI            0.000000

[7742 rows x 2 columns]

In [104]:
closeness=pd.DataFrame.from_dict(nx.closeness_centrality(G_graph), orient='index').reset_index()

closeness.rename(columns={'index': 'index', 0: 'Clossness Metric'})  



index  Clossness Metric
0            Rneifeld          0.049823
1             MSF_USA          0.043827
2         CarolLever6          0.000000
3       vondell_james          0.050951
4           mrsorokaa          0.059240
...               ...               ...
7737  RepTonyGonzales          0.000129
7738  DerrickBerthel1          0.000000
7739       AlexDunnin          0.000000
7740    Sweetescapes7          0.000000
7741    JOSEPHKAMOOTI          0.050951

[7742 rows x 2 columns]

In [135]:
## Importing train.csv data from part1
df= pd.read_csv("/Users/senanagblonon/Downloads/train.csv") 


In [138]:
##Creating New Columns features from part 1


df['com_follower_count'] = df.A_follower_count -  df.B_follower_count
df['com_following_count'] = df.A_following_count -  df.B_following_count
df['com_listed_count'] = df.A_listed_count -  df.B_listed_count
df['com_mentions_recieved'] = df.A_mentions_received-df.B_mentions_received
df['com_retweets_recieved'] = df.A_retweets_received -df.B_retweets_received
df['com_mentions_sent'] = df.A_mentions_sent - df.B_mentions_sent
df['com_retweets_sent'] = df.A_retweets_sent -  df.B_retweets_sent
df['com_posts'] = df.A_posts -  df.B_posts
df['com_network_feature_1'] = df.A_network_feature_1 - df.B_network_feature_1
df['com_network_feature_2'] = df.A_network_feature_2 - df.B_network_feature_2
df['com_network_feature_3'] = df.A_network_feature_3 - df.B_network_feature_3



In [137]:
## Dataset normalization
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()
scaler.fit(df)
scaled = scaler.fit_transform(df)
df1 = pd.DataFrame(scaled, columns=df.columns)


df1

Choice  A_follower_count  A_following_count  A_listed_count  \
0        0.0      5.801356e-06           0.000259        0.000005   
1        0.0      5.903975e-04           0.001011        0.000415   
2        0.0      1.995995e-04           0.001042        0.000184   
3        0.0      1.094596e-07           0.000006        0.000004   
4        1.0      1.247100e-03           0.000739        0.004809   
...      ...               ...                ...             ...   
5495     0.0      1.142457e-03           0.000159        0.002469   
5496     1.0      2.627029e-06           0.000208        0.000009   
5497     0.0      4.205710e-04           0.000577        0.001360   
5498     0.0      7.258318e-03           0.000179        0.001003   
5499     0.0      1.674731e-05           0.000790        0.000011   

      A_mentions_received  A_retweets_received  A_mentions_sent  \
0            4.221691e-07         0.000000e+00         0.000000   
1            7.889846e-05         5.896344e-05         0.073118   
2            2.218191e-05         2.169639e-05         0.068584   
3            6.627834e-06         4.056719e-07         0.016048   
4            1.298911e-04         8.466314e-05         0.362164   
...                   ...                  ...              ...   
5495         1.335590e-03         6.487868e-04         1.000000   
5496         1.174160e-06         0.000000e+00         0.000000   
5497         4.880555e-05         5.098698e-05         0.089243   
5498         5.516984e-04         1.049842e-03         0.069881   
5499         3.355992e-06         1.188747e-06         0.060872   

      A_retweets_sent   A_posts  A_network_feature_1  ...  \
0            0.000000  0.001356             0.000002  ...   
1            0.062425  0.026305             0.000401  ...   
2            0.030309  0.018081             0.000103  ...   
3            0.000000  0.014148             0.000007  ...   
4            0.199690  0.123131             0.000598  ...   
...               ...       ...                  ...  ...   
5495         0.326847  0.540691             0.003591  ...   
5496         0.000000  0.002605             0.000005  ...   
5497         0.014912  0.033183             0.000219  ...   
5498         0.000000  0.038335             0.002827  ...   
5499         0.047537  0.014783             0.000009  ...   

      com_following_count  com_listed_count  com_mentions_recieved  \
0                0.347553          0.510798               0.500196   
1                0.363888          0.511084               0.500224   
2                0.364108          0.512285               0.500213   
3                0.354055          0.512124               0.499956   
4                0.363242          0.514698               0.500258   
...                   ...               ...                    ...   
5495             0.363058          0.499415               0.497864   
5496             0.363577          0.512381               0.500203   
5497             0.363138          0.512916               0.500218   
5498             0.343235          0.491989               0.500402   
5499             0.362509          0.500375               0.500068   

      com_retweets_recieved  com_mentions_sent  com_retweets_sent  com_posts  \
0                  0.500128           0.447178           0.496772   0.482830   
1                  0.500151           0.516176           0.521014   0.510640   
2                  0.500143           0.531015           0.519267   0.507738   
3                  0.499684           0.329091           0.283996   0.424157   
4                  0.500170           0.637644           0.572905   0.550002   
...                     ...                ...                ...        ...   
5495               0.494701           0.852803           0.597587   0.722876   
5496               0.500132           0.500000           0.503991   0.497360   
5497               0.500153           0.501184           0.479779   0.505029  

In [139]:
## The feature importance from part 1, revealed that our top 4 predictors are:
#  com_network_feature_3
#  com_network_feature_2    
#  com_network_feature_1         
#  com_posts       

# Let choose weights such that bigger weights are respectively associated with most 
# important predictors and w1+w2+w3+w4=1
w1=0.40
w2=0.30
w3=0.20
w4=0.10


In [140]:
##Create score for each author from our twitter data
Score = w1 *df1['com_network_feature_3'] + w2 *df1['com_network_feature_2'] + w3 *df1['com_network_feature_1']+ w4 *df1['com_posts']

## Addiing score column to existing dataset
data["Score"]=Score



In [142]:
## Selecting 100 influencers from the tweets. 
data_100= data.nlargest(100,['Score'])
###create a list
# plist = data_100.values.tolist()
# print(plist)
data_100

Source           Target Type of Content     Score
773       Zee97052831      Zee97052831           Tweet  0.788161
3694    SpainOnAPlate  IsabelOakeshott       Non-Tweet  0.643085
3177       Rickardpmp    Conservatives       Non-Tweet  0.623416
1596   LloydDavidson8          YouTube       Non-Tweet  0.613037
3556         drmcrash     brawnyman713       Non-Tweet  0.612932
...               ...              ...             ...       ...
5260        imokedoke      margbrennan       Non-Tweet  0.535077
3140         _nika_ln   maryk_ProudDem       Non-Tweet  0.534834
3487      Lee_in_Iowa    IAPonomarenko       Non-Tweet  0.534834
2372     TheeRougarou       jojoptyoga       Non-Tweet  0.534737
98    pilarpaisajista  pilarpaisajista           Tweet  0.533953

[100 rows x 4 columns]

In [143]:
#Exporting file into csv
data_100.to_csv (r'C:\Users\senanagblonon\Downloads\data100.csv', index = False, header=True)
